In [2]:
import Pkg

In [4]:
Pkg.instantiate()

In [5]:
using LinearAlgebra
using NBInclude
@nbinclude("QOJulia_utils.ipynb")

proj_MB_Int (generic function with 1 method)

In [6]:
function calc_link_var(ψ1, ψ2)
    link_var = det(ψ1'*ψ2)
    return link_var/abs(link_var)
end

calc_link_var (generic function with 1 method)

In [7]:
function calc_link_var_svd(ψ1, ψ2)
    A = ψ1' * ψ2
    V, Σ, W = svd(A)
    M = V * W'
    ψ1 = ψ2 * M'
    link_var = det(ψ1'*ψ2)
    return link_var/abs(link_var)
end

calc_link_var_svd (generic function with 1 method)

In [7]:
function calc_link_vars(UUA, use_svd=true)
    N1,N2,__ = size(UUA)
    UU=zeros(Complex{Float64}, N1, N2, 2)
    dirs = [ [1, 0], [0,1] ]
    if use_svd == false
        for it1 in 1:N1
            for it2 in 1:N2
                for (idir, vdir) in enumerate(dirs)
                    UU[it1,it2,idir] = calc_link_var(UUA[it1,it2,:,:],
                                                     UUA[mod(it1-1+vdir[1],N1)+1,mod(it2-1+vdir[2],N2)+1,:,:])
                end
            end
        end
    elseif use_svd == true
        for it1 in 1:N1
            for it2 in 1:N2
                for (idir, vdir) in enumerate(dirs)
                    UU[it1,it2,idir] = calc_link_var_svd(UUA[it1,it2,:,:],
                                                     UUA[mod(it1-1+vdir[1],N1)+1,mod(it2-1+vdir[2],N2)+1,:,:])
                end
            end
        end
    end
    return UU 
end

calc_link_vars (generic function with 1 method)

In [8]:
function calc_F12(UU)
    F12 = log.(  UU[:,:,1].*
                circshift(UU[:,:,2],(-1,0))./
               (circshift(UU[:,:,1],(0,-1)).*
                UU[:,:,2]) 
            )
    return F12
end

calc_F12 (generic function with 1 method)

In [ ]:
function calc_chern_bands(UUA, band, use_svd=true)
    Chern = []
    F12list = []
    for band in bands
        UU = calc_link_vars(UUA[:,:,:,band[1]:band[2]], use_svd)
        F12 = calc_F12(UU)
        push!(F12list, F12)
        push!(Chern, sum(F12)/(2im*pi))
    end
    return Chern, F12list
end

In [12]:
function calc_all_states_H1(param_dict)
    
    model_H1 = param_dict["model_H1"]
    basis = param_dict["basis"]
    Tx = param_dict[param_dict["param_1"]]
    Ty = param_dict[param_dict["param_2"]]
    dimH = length(basis)
    N1 = length(Tx)
    N2 = length(Ty)
    EEA=zeros(N1, N2, dimH)
    UUA=zeros(Complex{Float64}, N1, N2, dimH, dimH)
    for it1 in 1:N1
        for it2 in 1:N2
            param_dict["θθ"]=[ Tx[it1], Ty[it2] ]
            H1_m = model_H1(param_dict)    
            #H1 = get_sp_op(basis, H1_m) #???
            
            # 4x2
            H1_m = (H1_m'+H1_m)/2
            #
            
            EEA[it1,it2,:], UUA[it1,it2,:,:] = eigen(H1_m)
        end
    end
    if "sub_range" in keys(param_dict)
        return EEA[:,:,param_dict["sub_range"]], UUA[:,:,:,param_dict["sub_range"]]
    else
        return EEA, UUA
    end
end

calc_all_states_H1 (generic function with 1 method)

In [15]:
function calc_all_states_H_mb(param_dict)
    
    Tx = param_dict[param_dict["param_1"]]
    Ty = param_dict[param_dict["param_2"]]
    H_mb = get_H_mb(param_dict)
    dimH_mb = param_dict["dimH_mb"]
    dimH_mb0 = param_dict["dimH_mb0"]
    N1 = length(Tx)
    N2 = length(Ty)
    EEA=zeros(N1, N2, dimH_mb)
    UUA=zeros(Complex{Float64}, N1, N2, dimH_mb0, dimH_mb0)
    for it1 in 1:N1
        for it2 in 1:N2
            param_dict["θθ"]=[ Tx[it1], Ty[it2] ]
            
            H_mb = get_H_mb(param_dict)
            #4x2
            H_mb = (H_mb'+H_mb)/2

            EE, UU = eigen(dense(H_mb).data)
            
            if "projection" in keys(param_dict)           
                P1M = param_dict["projection"][3].data
                PMB = zeros(ComplexF64,dimH_mb0,dimH_mb)
                for iMB in 1:dimH_mb0
                    i_occ_lists = Set( collect(permutations(param_dict["state_list0"][iMB])) )
                    ilistlen = length(i_occ_lists)
                    for jMB in 1:dimH_mb
                        j_occ_lists = Set( collect(permutations(param_dict["state_list"][jMB])) )
                        jlistlen = length(j_occ_lists)
                        for (iocc_ind,iocc) in enumerate(i_occ_lists)
                            for (jocc_ind,jocc) in enumerate(j_occ_lists)
                                P1val=1
                                for ind in 1:param_dict["PN"]
                                    P1val *= P1M[iocc[ind],jocc[ind]]
                                end    
                                PMB[iMB,jMB] += P1val/sqrt(ilistlen*jlistlen) 
                            end
                        end
                    end
                end
                EEA[it1,it2,:], UUA[it1,it2,:,:] = EE, PMB*UU*PMB' 
            else
                EEA[it1,it2,:], UUA[it1,it2,:,:] = EE, UU
            end
        end
    end

    return EEA, UUA
end

calc_all_states_Hmb (generic function with 1 method)

In [ ]:
function calc_all_states_H_mb_Finite_U(param_dict)
    
    Tx = param_dict[param_dict["param_1"]]
    Ty = param_dict[param_dict["param_2"]]
    dimH = param_dict["dimH_mb"]
    N1 = length(Tx)
    N2 = length(Ty)
    EEA=zeros(N1, N2, dimH)
    UUA=zeros(Complex{Float64}, N1, N2, dimH, dimH)
    for it1 in 1:N1
        for it2 in 1:N2
            param_dict["θθ"]=[ Tx[it1], Ty[it2] ]
            H_Finite, H_Int = get_Finite_U(param_dict)
            EEA[it1,it2,:], UUA[it1,it2,:,:] = eigen(dense(H_Finite).data)
        end
    end

    return EEA, UUA
end

In [ ]:
function calc_all_states_H_mb_Int(param_dict)
    
    Tx = param_dict[param_dict["param_1"]]
    Ty = param_dict[param_dict["param_2"]]
    dimH = param_dict["dimH_mb_int"]
    N1 = length(Tx)
    N2 = length(Ty)
    EEA=zeros(Complex{Float64},N1, N2, dimH)
    UUA=zeros(Complex{Float64}, N1, N2, dimH, dimH)
    for it1 in 1:N1
        for it2 in 1:N2
            param_dict["θθ"]=[ Tx[it1], Ty[it2] ]
            H_Total_Intt, basis_cut_mb = proj_MB_Int(param_dict)
            EEA[it1,it2,:], UUA[it1,it2,:,:] = eigen(dense(H_Total_Intt).data)
        end
    end
    return real(EEA), UUA
end

In [ ]:
function calc_all_states_H_mb_2(param_dict)
    
    Tx = param_dict[param_dict["param_1"]]
    Ty = param_dict[param_dict["param_2"]]
    dimH = param_dict["dimH_mb"]
    N1 = length(Tx)
    N2 = length(Ty)
    EEA=zeros(N1, N2, dimH)
    UUA=zeros(Complex{Float64}, N1, N2, dimH, dimH)
    for it1 in 1:N1
        for it2 in 1:N2
            param_dict["θθ"]=[ Tx[it1], Ty[it2] ]
            H_mb, UP = get_H_mb_2(param_dict)
            EE, UU = eigen(dense(H_mb).data)
            EEA[it1,it2,:], UUA[it1,it2,:,:] = EE, UP'.data*UU*UP.data 
        end
    end

    return EEA, UUA
end